In [ ]:
import pandas as pd

In [ ]:
import os
os.chdir(r"C:\Users\mukesk2\OneDrive - kochind.com\Tutorials\MLOPS\ML_Project1")
os.getcwd()

In [ ]:
data =pd.read_csv(r"C:\Users\mukesk2\OneDrive - kochind.com\Tutorials\MLOPS\ML_Project1\artifacts\data_ingestion\winequality-red.csv")
data.head()

In [ ]:
data.columns.tolist()

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    status_file: Path
    all_schema: dict



In [ ]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            status_file=Path(config.status_file),
            unzip_data_dir=Path(config.unzip_data_dir),    # <-- FIX
            all_schema=schema
        )
        return data_validation_config


In [ ]:
import os
from MLProject import logger


In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col in all_schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


In [ ]:
try:
    config=ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e
